## tensorflow_minist based CNN

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [101]:
mnist=input_data.read_data_sets("./mnist/data",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data\t10k-labels-idx1-ubyte.gz


In [107]:
tf.reset_default_graph()
x=tf.placeholder(tf.float32, [None,28,28,1]) #4차원
y=tf.placeholder(tf.float32, [None,10])
keep_prob=tf.placeholder(tf.float32)

In [108]:
# 그래프 정의
w1=tf.Variable(tf.random_normal([3,3,1,32])) # 32개의 특성을 가지고 합성곱 연산
# [3,3]:필터(커널)크기, 1:채널 특성 개수, 32:필터의 개수
L1=tf.nn.conv2d(x,w1,strides=[1,1,1,1], padding="SAME") #convolution
# strides=[1(고정),1(칸),1(줄),1(고정)], padding="SAME" : shape 유지
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
## X:[None,28,28,1]
## w1:[3,3,1,32]
## L1: CONV -> [None,28,28,32] 
## pooling -> [None, 14,14,32]
# pooling은 반드시 해야하는 것은 아니고, 용도에 맞은 풀링을 사용하면 됨
# ksize=kernel size [1(고정),적정한 값,적정한 값,1(고정)] => 보통 2칸씩 이동하긴하지만, 반드시 그럴 필요는 없음
# ksize가 커질수록 정확도가 떨어짐

w2=tf.Variable(tf.random_normal([3,3,32,64]))
# [3,3]:필터(커널)크기, 32:채널 특성 개수, 64:필터의 개수
L2=tf.nn.conv2d(L1,w2,strides=[1,1,1,1], padding="SAME") #convolution
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],padding="SAME")
# L2: CONV -> [None,14,14,64]
# pooling -> [None,7,7,64]

# FC 계층: 입력 7*7*64 => 출력 256
w3=tf.Variable(tf.random_normal([7*7*64, 256]))
L3=tf.reshape(L2,[-1,7*7*64]) # flatten:3차원 -> 1차원, -1 = None
L3=tf.matmul(L3,w3) # 결과 shape : [-1,256]
L3=tf.nn.relu(L3)
L3=tf.nn.dropout(L3, keep_prob)

w4=tf.Variable(tf.random_normal([256,10]))
model=tf.matmul(L3,w4) # [None,10], 결과값이 점수(score)로 나옴, 예측값

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
# score값이 확률값으로 변환되어, 원핫인코딩된 label과 곱해져 log를 취한 값이 출력되고 reduce_mean을 해주면
# softmax cost 값으로 결과 출력
optimizer=tf.train.AdamOptimizer(0.01).minimize(cost)


In [109]:
init = tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

for epoch in range(10):
    total_cost=0
    
    for i in range(total_batch): #600번(1에폭)
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape(-1,28,28,1)
        _,cv=sess.run([optimizer, cost], feed_dict={x:batch_xs, 
                                                    y:batch_ys, 
                                                    keep_prob:0.7})
        total_cost += cv
        
    print("에폭:", "%d" % (epoch+1),
          "평균 cost:","{:.3f}".format(total_cost/total_batch))
print("모델 생성 완료")
# 60000개 샘플 = 100 * 600

is_correct= tf.equal(tf.argmax(model,1),tf.argmax(y,1))
accuracy =tf.reduce_mean(tf.cast(is_correct,tf.float32))
print("정확도 : ", sess.run(accuracy, feed_dict={x:mnist.test.images.reshape(-1,28,28,1),
                                             y:mnist.test.labels,
                                             keep_prob:1}))

에폭: 1 평균 cost: 211.893
에폭: 2 평균 cost: 2.017
에폭: 3 평균 cost: 1.826
에폭: 4 평균 cost: 1.817
에폭: 5 평균 cost: 1.743
에폭: 6 평균 cost: 1.715
에폭: 7 평균 cost: 1.661
에폭: 8 평균 cost: 1.736
에폭: 9 평균 cost: 1.757
에폭: 10 평균 cost: 1.956
모델 생성 완료
정확도 :  0.4179


## tf.layers 서브 패키지 활용한 CNN 모델링

In [116]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

In [117]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# 기본적으로 inputs, outputs size, kernel_size 만 넣어주면
# 활성화 함수 적용은 물론, 컨볼루션 신경망을 만들기 위한 나머지 수치들은 알아서 계산해줍니다.
# 특히 Weights 를 계산하는데 xavier_initializer 를 쓰고 있는 등,
# 크게 신경쓰지 않아도 일반적으로 효율적인 신경망을 만들어줍니다.
L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


In [118]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          is_training: True})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.4f}'.format(total_cost / total_batch))

print('최적화 완료!')

#########
# 결과 확인
######
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   is_training: False}))

Epoch: 0001 Avg. cost = 0.1788
Epoch: 0002 Avg. cost = 0.0526
Epoch: 0003 Avg. cost = 0.0368
Epoch: 0004 Avg. cost = 0.0269
Epoch: 0005 Avg. cost = 0.0199
Epoch: 0006 Avg. cost = 0.0150
Epoch: 0007 Avg. cost = 0.0126
Epoch: 0008 Avg. cost = 0.0106
Epoch: 0009 Avg. cost = 0.0076
Epoch: 0010 Avg. cost = 0.0072
Epoch: 0011 Avg. cost = 0.0060
Epoch: 0012 Avg. cost = 0.0061
Epoch: 0013 Avg. cost = 0.0051
Epoch: 0014 Avg. cost = 0.0032
Epoch: 0015 Avg. cost = 0.0054
최적화 완료!
정확도: 0.9916


## keras에서 mnist CNN 모델링

In [64]:
from keras import layers
from keras import models

In [65]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3), activation='relu'))

In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [67]:
# 3차원으로 나온 결과를 1차원으로 출력해야함
model.add(layers.Flatten())

In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.add(layers.Dense(64,activation='relu'))

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
Total para

In [71]:
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [72]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [73]:
(train_images,train_labels),(test_images, test_labels)=mnist.load_data()

In [74]:
train_images.shape
train_images=train_images.reshape((60000, 28, 28,1)) # 채널(색상) 추가
train_images=train_images.astype('float32')/255
test_images=test_images.reshape((10000, 28, 28,1)) # 채널(색상) 추가
test_images=test_images.astype('float32')/255

In [75]:
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

In [76]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

W0819 15:09:25.395745  2452 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [77]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

W0819 15:09:52.930166  2452 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0819 15:09:53.015935  2452 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
60000/60000 [==============================] - 28s 460us/step - loss: 0.1773 - acc: 0.94501s
Epoch 2/5
60000/60000 [==============================] - 27s 450us/step - loss: 0.0465 - acc: 0.9859
Epoch 3/5
60000/60000 [==============================] - 28s 459us/step - loss: 0.0326 - acc: 0.9903
Epoch 4/5
60000/60000 [==============================] - 27s 448us/step - loss: 0.0248 - acc: 0.9921
Epoch 5/5
60000/60000 [==============================] - 27s 445us/step - loss: 0.0192 - acc: 0.9942


In [78]:
test_loss, test_acc=model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 127us/step


0.9912

In [92]:
import os, shutil

# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = 'data/dogs-vs-cats/train/'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = 'data/dogs-vs-cats/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst) # src에 있는걸 dst로 복사하는 함수

# 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [93]:
len(os.listdir(train_cats_dir))
# 트레이닝 고양이, 강아지 각각 1000개

1000

### mission : 고양이, 강아지 분류기 제작
#### 내일 : 연관규칙(apriori)